# Part 3: 從 Xarray 到 ML Pipeline (50 min)

這個 notebook 是整個 workshop 的核心，展示如何將大型 N-D array 資料無縫整合到機器學習流程中。

## 本節內容

1. **定義 ML 任務**：對流分類（Convection Classification）
2. **建立 Labels**：從 CAPE 定義對流事件
3. **xbatcher**：產生訓練用的時空 patches
4. **PyTorch 整合**：使用 `xbatcher.loaders.torch.MapDataset`
5. **模型訓練**：簡單的 CNN 分類器
6. **空間驗證**：使用 xskillscore 評估預測品質

---

## 學習目標

- 理解「時空 batch」的概念
- 掌握 xbatcher 的兩階段設計（BatchGenerator → MapDataset）
- 正確設定 DataLoader 參數（batch_size=None！）
- 從 Xarray 到 PyTorch Tensor 的資料流
- 保留空間資訊進行驗證（vs 丟棄空間資訊的傳統 ML）

## 0. 環境準備

In [ ]:
import dask
from dask.distributed import Client
import xarray as xr
import xbatcher
import intake
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# 啟動 Dask Client
client = Client(processes=False, n_workers=2, threads_per_worker=2, memory_limit='2GB')
print(f"Dask Dashboard: {client.dashboard_link}")

# 載入 catalog
catalog = intake.open_catalog('catalog.yaml')

## 1. 定義 ML 任務：對流分類

### 什麼是對流分類？

對流（convection）是指空氣垂直運動導致的劇烈天氣現象：
- 雷暴（thunderstorms）
- 強降雨
- 冰雹、龍捲風

預報對流非常重要，但傳統數值模式的解析度不足。我們希望用 ML 從大尺度變數預測小尺度對流。

### 任務定義

**輸入（Features）**：
- CAPE（Convective Available Potential Energy）：對流可用位能
- CIN（Convective Inhibition）：對流抑制
- K-index：不穩定指數
- BLH（Boundary Layer Height）：邊界層高度

**輸出（Label）**：
- 二元分類：是否發生對流（0 or 1）

### Label 的定義

實務上，對流的定義可以是：
- 雷達回波 > 40 dBZ（需要雷達資料）
- 降雨強度 > 10 mm/hr（需要雨量資料）
- **簡化版**：CAPE > 某個閾值 + CIN < 某個閾值

這個 workshop 我們用簡化版，重點是展示流程，不是預報精度。

## 2. 準備資料與建立 Labels

### 2.1 載入資料

In [ ]:
# 載入 2019 年資料（或使用前面儲存的優化版）
ds = catalog.era5_2019.to_dask()

# Resample 到 daily 以減少資料量
# 對於 ML，我們通常不需要 hourly 解析度
ds_daily = ds.resample(time='1D').mean()

print("Dataset:")
print(ds_daily)
print()
print(f"Shape: {ds_daily['cape'].shape}")
print(f"Total size: {ds_daily.nbytes / 1e9:.2f} GB")

### 2.2 建立對流 Label

我們定義對流發生的條件：
- CAPE > 1000 J/kg（有足夠的不穩定能量）
- CIN > -50 J/kg（抑制不會太強）

這個閾值是簡化的，實務上需要根據當地氣候調整。

In [ ]:
# 建立 binary label
convection_flag = (
    (ds_daily['cape'] > 1000) & 
    (ds_daily['cin'] > -50)
).astype(np.float32)  # 轉為 float32 以便與 features 相容

# 加入 Dataset
ds_daily['convection_flag'] = convection_flag

print("Convection flag:")
print(ds_daily['convection_flag'])
print()

# 檢查 class balance
flag_mean = convection_flag.mean().compute()
print(f"Convection occurrence rate: {flag_mean.values * 100:.2f}%")
print(f"  Class 0 (no convection): {(1 - flag_mean.values) * 100:.2f}%")
print(f"  Class 1 (convection): {flag_mean.values * 100:.2f}%")

### Class Imbalance

如果對流發生率很低（例如 < 10%），這是典型的 imbalanced classification problem。

處理方法（本 workshop 不深入實作）：
1. **Weighted loss**：給少數類更高權重
2. **Oversampling**：多採樣對流事件
3. **Focal loss**：專注於難分類的樣本

這裡我們先用簡單的 cross-entropy loss，重點是流程。

### 2.3 選取特定區域與時間段

為了加速示範，我們選取：
- 時間：2019 年 6-8 月（夏季，對流旺盛）
- 空間：20-30°N, 110-125°E（華南地區）

In [ ]:
# 選取子集
ds_subset = ds_daily.sel(
    time=slice('2019-07-01', '2019-07-15')  # Demo: 只用半個月,
    latitude=slice(22, 28)  # Demo: 更小區域,
    longitude=slice(115, 122)  # Demo: 更小區域
)

print("Subset for training:")
print(ds_subset)
print()
print(f"Time steps: {len(ds_subset.time)}")
print(f"Spatial shape: {len(ds_subset.latitude)} x {len(ds_subset.longitude)}")
print(f"Total size: {ds_subset.nbytes / 1e6:.2f} MB")

## 3. 資料分割：Train / Val / Test

### 時間序列分割的重要性

對於時間序列資料，**不能隨機分割**！

原因：
- 相鄰時間點高度相關（temporal autocorrelation）
- 隨機分割會「洩露未來資訊」到訓練集
- 模型會過度擬合短期變化

正確做法：**按時間順序分割**
- Training: 前 70%
- Validation: 中間 15%
- Test: 最後 15%

In [ ]:
# 計算分割點
n_total = len(ds_subset.time)
n_train = int(n_total * 0.7)
n_val = int(n_total * 0.15)

# 時間序列分割
train_ds = ds_subset.isel(time=slice(0, n_train))
val_ds = ds_subset.isel(time=slice(n_train, n_train + n_val))
test_ds = ds_subset.isel(time=slice(n_train + n_val, None))

print("Data split:")
print(f"  Training: {len(train_ds.time)} days ({train_ds.time.values[0]} to {train_ds.time.values[-1]})")
print(f"  Validation: {len(val_ds.time)} days ({val_ds.time.values[0]} to {val_ds.time.values[-1]})")
print(f"  Test: {len(test_ds.time)} days ({test_ds.time.values[0]} to {test_ds.time.values[-1]})")

## 4. xbatcher：產生時空 Patches

### 什麼是時空 Batch？

傳統 ML 的 batch：
- 從 N 個樣本中隨機選 B 個
- 每個樣本是獨立的 feature vector

時空 batch：
- 從 3D/4D array 中切出小的「patches」
- 每個 patch 包含時間和空間維度
- 保留時空結構（對 CNN/RNN 很重要）

### xbatcher 的設計

xbatcher 提供兩階段流程：

**Stage 1: BatchGenerator**
- 定義如何從 Xarray Dataset 切出 batches
- 指定 `input_dims`（空間大小）和 `batch_dims`（時間大小）
- 仍然是 **lazy** 的（不會實際讀取資料）

**Stage 2: MapDataset**
- 將 BatchGenerator 包裝成 PyTorch Dataset
- 處理從 Xarray → NumPy → Tensor 的轉換
- 可以搭配 DataLoader 做 shuffling、multiprocessing

### 4.1 Stage 1: 創建 BatchGenerator

我們需要**分別**為 features 和 labels 創建 BatchGenerator。

#### 參數說明

- **input_dims**: 空間維度的大小（不會沿著這些維度切分）
  - 例如 `{'latitude': 16, 'longitude': 16}` 表示每個 patch 是 16x16
  - 如果資料有 40 個 latitude 點，會產生 40/16 = 2.5 → 3 個 patches（有 overlap）

- **batch_dims**: 會切分的維度（通常是時間）
  - 例如 `{'time': 32}` 表示每個 batch 包含 32 個時間步
  - 如果資料有 100 天，會產生 100/32 ≈ 3 個 batches

- **preload_batch**: 是否預先載入整個 batch 到記憶體
  - `False`（推薦）：保持 lazy，只在迭代時載入
  - `True`：會預先 .compute()，可能記憶體不足

In [ ]:
# 定義 feature 變數
feature_vars = ['cape', 'cin', 'k_index', 'blh']

# Stage 1a: 為 features 創建 BatchGenerator
X_bgen = xbatcher.BatchGenerator(
    train_ds[feature_vars],
    input_dims={'latitude': 12, 'longitude': 12}  # Demo: 更小 patch,  # 16x16 空間 patches
    batch_dims={'time': 16}  # Demo: 更小 batch,                       # 32 time steps per batch
    preload_batch=False  # 保持 lazy evaluation
)

# Stage 1b: 為 labels 創建 BatchGenerator
y_bgen = xbatcher.BatchGenerator(
    train_ds['convection_flag'],
    input_dims={'latitude': 12, 'longitude': 12}  # Demo: 更小 patch,
    batch_dims={'time': 16}  # Demo: 更小 batch,
    preload_batch=False
)

print("BatchGenerators created:")
print(f"  X_bgen: {len(list(X_bgen))} batches")
print(f"  y_bgen: {len(list(y_bgen))} batches")
print()
print("Note: 上面的 list() 會實際迭代，只是為了計數。")
print("      實際使用時不需要這樣做。")

### 4.2 檢視單一 Batch 的結構

In [ ]:
# 重新創建（因為 generator 已經被消耗了）
X_bgen = xbatcher.BatchGenerator(
    train_ds[feature_vars],
    input_dims={'latitude': 12, 'longitude': 12}  # Demo: 更小 patch,
    batch_dims={'time': 16}  # Demo: 更小 batch,
    preload_batch=False
)

# 取得第一個 batch
first_batch = next(iter(X_bgen))

print("First batch (still lazy):")
print(first_batch)
print()
print(f"Dimensions: {first_batch.dims}")
print(f"Shape: {first_batch.dims}")
print(f"Variables: {list(first_batch.data_vars)}")
print()
print(f"CAPE shape in this batch: {first_batch['cape'].shape}")
print(f"Type: {type(first_batch['cape'].data)}")

### 理解 Batch Shape

原始資料：`(time, latitude, longitude)`
- time: 例如 64 天
- latitude: 40 點
- longitude: 60 點

經過 xbatcher：`(time, latitude, longitude)`
- time: 32（batch_dims）
- latitude: 16（input_dims）
- longitude: 16（input_dims）

這個 shape 會被送入 CNN。

## 5. Stage 2: PyTorch 整合

### 5.1 使用 xbatcher.loaders.torch.MapDataset

**重要**：這是 xbatcher 官方提供的 PyTorch 整合方式。

不要自己寫 `Dataset` wrapper！xbatcher 已經處理好了：
- Xarray → NumPy 轉換
- NumPy → Tensor 轉換
- 維度順序調整（Xarray 是 (time, lat, lon)，PyTorch 慣例是 (batch, channel, height, width)）
- Lazy loading 管理

In [ ]:
# 重新創建 BatchGenerators（確保沒被消耗）
X_bgen = xbatcher.BatchGenerator(
    train_ds[feature_vars],
    input_dims={'latitude': 12, 'longitude': 12}  # Demo: 更小 patch,
    batch_dims={'time': 16}  # Demo: 更小 batch,
    preload_batch=False
)

y_bgen = xbatcher.BatchGenerator(
    train_ds['convection_flag'],
    input_dims={'latitude': 12, 'longitude': 12}  # Demo: 更小 patch,
    batch_dims={'time': 16}  # Demo: 更小 batch,
    preload_batch=False
)

# Stage 2: 使用官方的 MapDataset
train_dataset = xbatcher.loaders.torch.MapDataset(
    X_bgen,  # features
    y_bgen   # labels
)

print(f"PyTorch Dataset created: {len(train_dataset)} batches")
print(f"Type: {type(train_dataset)}")

### 5.2 檢視 Dataset 回傳的資料

MapDataset 回傳 `(X, y)` tuple，其中：
- X: Tensor of shape `(batch_dim, n_features, height, width)`
- y: Tensor of shape `(batch_dim, height, width)`

In [ ]:
# 取得一個樣本
X_sample, y_sample = train_dataset[0]

print("Sample from Dataset:")
print(f"  X type: {type(X_sample)}")
print(f"  X shape: {X_sample.shape}")
print(f"  X dtype: {X_sample.dtype}")
print()
print(f"  y type: {type(y_sample)}")
print(f"  y shape: {y_sample.shape}")
print(f"  y dtype: {y_sample.dtype}")
print()
print("Shape interpretation:")
print(f"  X: (time={X_sample.shape[0]}, features={X_sample.shape[1]}, lat={X_sample.shape[2]}, lon={X_sample.shape[3]})")
print(f"  y: (time={y_sample.shape[0]}, lat={y_sample.shape[1]}, lon={y_sample.shape[2]})")

### 5.3 創建 DataLoader

**關鍵參數**：`batch_size=None`

為什麼？
- xbatcher 已經定義了 batch（透過 `batch_dims`）
- DataLoader 的 `batch_size` 是用來「把多個樣本組成一個 batch」
- 但我們的「一個樣本」已經是一個 batch（32 time steps）
- 如果設定 `batch_size=4`，會變成 `(4, 32, 4, 16, 16)`（錯誤！）

**正確設定**：
```python
DataLoader(dataset, batch_size=None, ...)
```

這樣 DataLoader 只負責：
- Shuffling（如果需要）
- Multiprocessing（num_workers）
- 不會改變 batch 的 shape

In [ ]:
# 創建 DataLoader
train_loader = DataLoader(
    train_dataset,
    batch_size=None,  # 不要再增加 batch 維度！
    shuffle=True,     # 打亂 batches 順序（不是打亂 batch 內的順序）
    num_workers=2,    # 平行載入資料
    multiprocessing_context='forkserver'  # 避免 Dask client pickle 問題
)

print(f"DataLoader created: {len(train_loader)} batches")
print()
print("Parameters:")
print(f"  batch_size: None (xbatcher already defines batch)")
print(f"  shuffle: True")
print(f"  num_workers: 2")

### 5.4 測試 DataLoader

In [ ]:
# 迭代取得一個 batch
for X_batch, y_batch in train_loader:
    print("Batch from DataLoader:")
    print(f"  X: {X_batch.shape}, dtype: {X_batch.dtype}")
    print(f"  y: {y_batch.shape}, dtype: {y_batch.dtype}")
    print()
    print(f"  X min/max: {X_batch.min():.2f} / {X_batch.max():.2f}")
    print(f"  y unique values: {torch.unique(y_batch)}")
    break  # 只看第一個 batch

## 6. 定義 CNN 模型

我們使用一個簡單的 3D CNN（時空 convolution）：
- Input: `(batch, channels, time, height, width)` = (32, 4, 32, 16, 16)
- Output: `(batch, time, height, width)` = (32, 32, 16, 16)

模型架構：
1. Conv3D + ReLU + MaxPool
2. Conv3D + ReLU + MaxPool  
3. Conv3D（output layer，sigmoid 激活）

這只是示範，不是 state-of-the-art。

In [ ]:
class SimpleConvectionCNN(nn.Module):
    def __init__(self, in_channels=4):
        super().__init__()
        
        # 3D Convolutions (time + space)
        self.conv1 = nn.Conv3d(in_channels, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(32, 1, kernel_size=3, padding=1)  # output: 1 channel
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # for binary classification
        
    def forward(self, x):
        # x: (batch, channels, time, height, width)
        # 但我們的資料是 (time, channels, height, width)
        # 需要調整維度順序
        
        # Permute: (time, channels, height, width) -> (1, channels, time, height, width)
        # 加上 batch 維度（因為 batch_size=None，沒有 batch 維度）
        if x.dim() == 4:
            x = x.unsqueeze(0)  # add batch dim
        
        # Permute: (batch, time, channels, height, width) -> (batch, channels, time, height, width)
        x = x.permute(0, 2, 1, 3, 4)
        
        # Convolution layers
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.sigmoid(self.conv3(x))
        
        # Output: (batch, 1, time, height, width)
        # Squeeze channel dim and permute back
        x = x.squeeze(1)  # (batch, time, height, width)
        
        # Remove batch dim if added
        if x.size(0) == 1:
            x = x.squeeze(0)  # (time, height, width)
        
        return x

# 創建模型
model = SimpleConvectionCNN(in_channels=4)
print(model)
print()

# 計算參數數量
n_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {n_params:,}")

### 測試模型 Forward Pass

In [ ]:
# 創建 dummy input
dummy_input = torch.randn(32, 4, 16, 16)  # (time, channels, height, width)

# Forward pass
with torch.no_grad():
    output = model(dummy_input)

print(f"Input shape: {dummy_input.shape}")
print(f"Output shape: {output.shape}")
print(f"Output range: [{output.min():.3f}, {output.max():.3f}]")
print()
print("✓ Model forward pass successful!")

## 7. 訓練迴圈

### 7.1 設定訓練參數

In [ ]:
# 設定 device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = model.to(device)

# Loss function
criterion = nn.BCELoss()  # Binary Cross Entropy

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training config
n_epochs = 2  # Demo: 快速展示流程

print(f"Training configuration:")
print(f"  Epochs: {n_epochs}")
print(f"  Optimizer: Adam (lr=0.001)")
print(f"  Loss: Binary Cross Entropy")

### 7.2 訓練迴圈

In [ ]:
# Training loop
history = {'loss': []}

for epoch in range(n_epochs):
    model.train()
    epoch_loss = 0.0
    n_batches = 0
    
    for X_batch, y_batch in train_loader:
        # Move to device
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        # Forward pass
        outputs = model(X_batch)
        
        # Calculate loss
        loss = criterion(outputs, y_batch)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Record
        epoch_loss += loss.item()
        n_batches += 1
    
    # Epoch summary
    avg_loss = epoch_loss / n_batches
    history['loss'].append(avg_loss)
    
    print(f"Epoch {epoch+1}/{n_epochs} - Loss: {avg_loss:.4f}")

print("\n✓ Training complete!")

### 7.3 繪製 Training Loss

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, n_epochs+1), history['loss'], marker='o', linewidth=2, markersize=8)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss', fontsize=13)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. 模型評估與視覺化

### 8.1 在 Test Set 上預測

In [ ]:
# 為 test set 創建 DataLoader
X_test_bgen = xbatcher.BatchGenerator(
    test_ds[feature_vars],
    input_dims={'latitude': 16, 'longitude': 16},
    batch_dims={'time': 32},
    preload_batch=False
)

y_test_bgen = xbatcher.BatchGenerator(
    test_ds['convection_flag'],
    input_dims={'latitude': 16, 'longitude': 16},
    batch_dims={'time': 32},
    preload_batch=False
)

test_dataset = xbatcher.loaders.torch.MapDataset(X_test_bgen, y_test_bgen)
test_loader = DataLoader(
    test_dataset,
    batch_size=None,
    shuffle=False,  # test set 不 shuffle
    num_workers=2,
    multiprocessing_context='forkserver'
)

print(f"Test set: {len(test_loader)} batches")

In [ ]:
# Evaluation
model.eval()
test_loss = 0.0
predictions = []
targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        
        test_loss += loss.item()
        predictions.append(outputs.cpu())
        targets.append(y_batch.cpu())

avg_test_loss = test_loss / len(test_loader)
print(f"Test Loss: {avg_test_loss:.4f}")

# Concatenate all predictions
predictions = torch.cat(predictions, dim=0)
targets = torch.cat(targets, dim=0)

print(f"\nPredictions shape: {predictions.shape}")
print(f"Targets shape: {targets.shape}")

### 8.2 計算分類指標

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 轉為 binary predictions (threshold = 0.5)
pred_binary = (predictions > 0.5).float()

# Flatten for sklearn
pred_flat = pred_binary.flatten().numpy()
target_flat = targets.flatten().numpy()

# Calculate metrics
accuracy = accuracy_score(target_flat, pred_flat)
precision = precision_score(target_flat, pred_flat, zero_division=0)
recall = recall_score(target_flat, pred_flat, zero_division=0)
f1 = f1_score(target_flat, pred_flat, zero_division=0)

print("Classification Metrics:")
print(f"  Accuracy:  {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall:    {recall:.4f}")
print(f"  F1 Score:  {f1:.4f}")

### 8.3 視覺化：預測 vs 真實

In [ ]:
# 選取一個時間步驟和空間 patch 來視覺化
t_idx = 10  # 第 10 個時間步

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# True labels
im1 = axes[0].imshow(targets[t_idx], cmap='RdYlBu_r', vmin=0, vmax=1)
axes[0].set_title('Ground Truth', fontsize=12)
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')
plt.colorbar(im1, ax=axes[0])

# Predictions (probability)
im2 = axes[1].imshow(predictions[t_idx], cmap='RdYlBu_r', vmin=0, vmax=1)
axes[1].set_title('Predicted Probability', fontsize=12)
axes[1].set_xlabel('Longitude')
plt.colorbar(im2, ax=axes[1])

# Binary predictions
im3 = axes[2].imshow(pred_binary[t_idx], cmap='RdYlBu_r', vmin=0, vmax=1)
axes[2].set_title('Binary Prediction (>0.5)', fontsize=12)
axes[2].set_xlabel('Longitude')
plt.colorbar(im3, ax=axes[2])

plt.tight_layout()
plt.show()

## 9. 使用 xskillscore 進行空間驗證

### 為什麼需要 xskillscore？

傳統 ML 評估（accuracy, F1）把所有像素當作獨立樣本，忽略了：
- **空間連續性**：相鄰格點的預測應該平滑
- **空間尺度**：大範圍的錯誤 vs 小範圍的錯誤
- **空間相關**：預測的空間結構是否合理？

xskillscore 提供「空間感知」的驗證指標：
- **Spatial correlation**：預測場與真實場的空間相關
- **RMSE by region**：不同區域的誤差
- **Fractions Skill Score**：考慮空間鄰域的驗證

### 為什麼 xskillscore 適合這個 workflow？

xskillscore 直接接受 **Xarray DataArray**：
- 保留經緯度座標
- 可以做區域加權（area-weighted metrics）
- 可以沿著不同維度計算（時間、空間、ensemble）

這就是為什麼我們用 Xarray 而不是 pandas/NumPy 的原因之一。

### 9.1 將預測轉回 Xarray

我們需要把 PyTorch Tensor 轉回 Xarray，恢復座標資訊。

In [ ]:
# 注意：這裡是簡化版，實務上需要正確對應每個 patch 的座標
# 為了示範，我們假設 predictions 和 test_ds 的空間範圍相同

# 取得一個 batch 的座標
sample_batch = next(iter(X_test_bgen))
time_coords = sample_batch['time'].values
lat_coords = sample_batch['latitude'].values
lon_coords = sample_batch['longitude'].values

# 創建 Xarray DataArray
pred_da = xr.DataArray(
    predictions[:len(time_coords)].numpy(),  # 限制到實際的時間長度
    dims=['time', 'latitude', 'longitude'],
    coords={
        'time': time_coords,
        'latitude': lat_coords,
        'longitude': lon_coords
    },
    name='convection_probability'
)

target_da = xr.DataArray(
    targets[:len(time_coords)].numpy(),
    dims=['time', 'latitude', 'longitude'],
    coords={
        'time': time_coords,
        'latitude': lat_coords,
        'longitude': lon_coords
    },
    name='convection_truth'
)

print("Predictions as Xarray:")
print(pred_da)
print()
print("Targets as Xarray:")
print(target_da)

### 9.2 計算空間相關係數

In [ ]:
import xskillscore as xs

# 計算每個時間步的空間相關
spatial_corr = xs.pearson_r(pred_da, target_da, dim=['latitude', 'longitude'])

print("Spatial correlation (per time step):")
print(spatial_corr.values)
print()
print(f"Mean spatial correlation: {spatial_corr.mean().values:.4f}")
print(f"Std: {spatial_corr.std().values:.4f}")

# 繪圖
plt.figure(figsize=(10, 4))
spatial_corr.plot(marker='o')
plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
plt.title('Spatial Correlation over Time', fontsize=13)
plt.ylabel('Pearson r', fontsize=12)
plt.xlabel('Time', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 9.3 計算空間 RMSE

In [ ]:
# 計算 RMSE
rmse = xs.rmse(pred_da, target_da, dim=['time', 'latitude', 'longitude'])

print(f"Overall RMSE: {rmse.values:.4f}")

# 也可以計算每個格點的時間 RMSE
rmse_spatial = xs.rmse(pred_da, target_da, dim='time')

plt.figure(figsize=(10, 6))
rmse_spatial.plot(cmap='YlOrRd', vmin=0)
plt.title('RMSE by Location (averaged over time)', fontsize=13)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("紅色區域：模型預測誤差較大")
print("黃色/綠色：預測較準確")
print("可以幫助識別模型在哪些地理位置表現較差")

## 10. 完整 Workflow 回顧

讓我們回顧整個從「大型 N-D array」到「ML 模型」的流程：

```
1. 資料儲存
   Zarr (on-disk, chunked)
   ↓
   
2. 資料讀取
   intake-xarray + Dask
   ↓ (lazy)
   
3. 前處理
   - Resample (hourly → daily)
   - 建立 labels
   - 時間序列分割
   ↓ (still lazy)
   
4. Batch 生成
   xbatcher.BatchGenerator
   ↓
   
5. PyTorch 整合
   xbatcher.loaders.torch.MapDataset
   ↓
   
6. 資料載入
   DataLoader (batch_size=None)
   ↓ (now eager, on-demand)
   
7. 模型訓練
   PyTorch training loop
   ↓
   
8. 預測與評估
   - 傳統 metrics (accuracy, F1)
   - 空間 metrics (xskillscore)
   ↓
   
9. 結果視覺化
   Xarray + matplotlib
```

### 關鍵設計原則

1. **Lazy as long as possible**
   - 直到 DataLoader 迭代時才實際讀取資料
   - 減少記憶體佔用

2. **保留元資料**
   - 使用 Xarray 而不是 NumPy
   - 座標資訊對驗證和視覺化很重要

3. **批次處理**
   - xbatcher 自動處理時空切分
   - 不需要手動管理索引

4. **平行化**
   - Dask 處理資料讀取的平行化
   - DataLoader 的 num_workers 處理前處理平行化
   - GPU 處理模型訓練平行化

## 11. 常見問題與除錯

### Q1: DataLoader 報錯 "batch_size should be None"

**原因**：xbatcher 已經定義了 batch，不應該再用 DataLoader 的 batch_size。

**解法**：
```python
DataLoader(dataset, batch_size=None, ...)  # 正確
DataLoader(dataset, batch_size=4, ...)     # 錯誤！
```

### Q2: Multiprocessing 報錯 "cannot pickle Client"

**原因**：Dask Client 無法被 pickle，但 DataLoader 的 multiprocessing 需要 pickle。

**解法**：
```python
DataLoader(..., multiprocessing_context='forkserver')  # 使用 forkserver
# 或
DataLoader(..., num_workers=0)  # 不使用 multiprocessing
```

### Q3: 記憶體不足（OOM）

**原因**：
- Batch 太大（時間或空間維度）
- preload_batch=True
- num_workers 太多

**解法**：
1. 減小 batch_dims 或 input_dims
2. 確保 preload_batch=False
3. 減少 num_workers
4. 調整 Dask Client 的 memory_limit

### Q4: 訓練很慢

**可能原因**：
- I/O 瓶頸：增加 num_workers
- Chunk 太小：考慮 rechunk
- CPU 計算：檢查是否正確使用 GPU

**除錯**：觀察 Dask Dashboard，看時間花在哪裡。

## 12. 延伸方向

這個 workshop 展示了基礎流程，實務上可以延伸：

### 資料面
- 加入更多變數（濕度、風場、溫度剖面）
- 多資料來源融合（ERA5 + 衛星 + 地面觀測）
- 時間滯後特徵（t-1, t-2 小時的資料）

### 模型面
- 更複雜的架構（UNet, ResNet, Transformer）
- 序列模型（LSTM, GRU）用於時間序列
- Ensemble 模型

### 訓練面
- Class imbalance 處理（weighted loss, focal loss）
- Data augmentation（spatial flip, rotation）
- Transfer learning（預訓練模型）

### 驗證面
- 更多空間指標（Fractions Skill Score, SAL）
- Case study（分析特定事件）
- 區域化評估（山區 vs 平地）

## 13. 總結

完成這個 notebook 後，你應該能夠：

- [ ] 定義適合 ML 的氣象任務
- [ ] 建立有意義的 labels
- [ ] 使用 xbatcher 產生時空 batches
- [ ] 正確整合 xbatcher 與 PyTorch
- [ ] 設定 DataLoader（batch_size=None!）
- [ ] 訓練一個簡單的 CNN 模型
- [ ] 使用 xskillscore 進行空間驗證
- [ ] 理解完整的 out-of-core ML workflow

### 核心要點

1. **xbatcher 的兩階段設計是關鍵**
   - BatchGenerator → MapDataset
   - 不要自己寫 Dataset wrapper

2. **batch_size=None 避免維度錯誤**
   - xbatcher 已經定義 batch
   - DataLoader 只負責 shuffling 和 multiprocessing

3. **保留空間資訊很重要**
   - 轉回 Xarray 做驗證
   - 空間相關性是氣象預報的核心

4. **Lazy evaluation 貫穿整個流程**
   - 從 Zarr 讀取到 DataLoader 都是 lazy
   - 只在需要時才載入資料

這個工作流程可以擴展到：
- 更大的資料集（TB 級別）
- 更複雜的模型（深度學習）
- 分散式訓練（多 GPU、多節點）

重點是**理解原理**，而不是記住 API。

In [ ]:
# 關閉 Dask Client
# client.close()

print("Workshop completed! 🎉")